In [1]:
trg_path = './sample/'

In [2]:
import glob
import os

zipf_path = []

In [3]:
for filename in glob.glob(os.path.join(trg_path,'**/*.zip'), recursive=True):
    zipf_path.append(filename)
    print(zipf_path)

['./sample\\새파일.zip']


In [4]:
import zipfile

In [5]:
for zipf in zipf_path:
    with zipfile.ZipFile(zipf) as myzip:
        zipinfo = myzip.infolist()
        for info in zipinfo:
            decode_name = info.filename  #.encode('cp437').decode('euc-kr') #한글 깨짐 방지
            info.filename = os.path.join(trg_path, decode_name)
            myzip.extract(info)

In [7]:
import openpyxl as op

In [13]:
def getFileName(trg_path):
    forderlist = os.listdir(trg_path)

    wb = op.Workbook()

    ws= wb.active

    i=2

    current_folder = trg_path

    filelist= os.listdir(current_folder)

    for fname in filelist:
        ws.cell(row=i, column=1).value = current_folder + '/'
        ws.cell(row=i, column=2).value = fname
        i+=1

    ws.cell(row=1, column=1).value = '파일경로'
    ws.cell(row=1, column=2).value = '파일명(변경전)'
    ws.cell(row=1, column=3).value = '파일명(변경후)'

    wb.save(os.path.join(trg_path, 'filelist.xlsx'))


In [14]:
getFileName(trg_path)

In [15]:
def excelRead(filepath:str) -> tuple:
    wb = op.load_workbook(filepath)

    ws = wb.active

    folderpath = [r[0].value for r in ws]
    file_before = [r[1].value for r in ws]
    file_after = [r[2].value for r in ws]

    len_num = len(folderpath)
    datalist = []
    for i in range(1, len_num):
        temp_tuple = (folderpath[i], file_before[i], file_after[i])
        datalist.append(temp_tuple)

    return datalist

In [16]:
rename_list = excelRead(os.path.join(trg_path, 'filelist.xlsx'))
print(rename_list)

[('./sample//', 'sample', 'sample'), ('./sample//', '새파일.tar', '새파일.tar'), ('./sample//', '새파일.txt', '새파일1.txt'), ('./sample//', '새파일.zip', '새파일.zip'), ('./sample//', '새파일2.txt', '새파일2.txt')]


In [17]:
import shutil

In [24]:
def fileRename(datalist : list):
    for data in datalist:
        print(data[1]+'의 파일명을 '+data[2]+'로 변경합니다.')
        shutil.move(data[0]+data[1], data[0]+data[2])

In [25]:
fileRename(rename_list)

sample의 파일명을 sample로 변경합니다.
새파일.tar의 파일명을 새파일.tar로 변경합니다.
새파일.txt의 파일명을 새파일1.txt로 변경합니다.
새파일.zip의 파일명을 새파일.zip로 변경합니다.
새파일2.txt의 파일명을 새파일2.txt로 변경합니다.


In [26]:
import fnmatch

In [45]:
def categoryList(trg_path:str)->list:
    file_list = []
    for filename in os.listdir(trg_path):
        if fnmatch.fnmatch(filename, '새??_[0-9].*'):
            file_list.append(filename)
    category = []

    for file in file_list:
        temp_list = file.split("_")
        category.append(temp_list[0])

    temp_set = set(category)
    result = list(temp_set)

    return result

In [46]:
categorylist = categoryList(trg_path)+['기타']
print(categorylist)

['새파일', '기타']


In [47]:
import pathlib

In [48]:
clean_path = './clean_folder'
def makeFolder(clean_path:str, categorylist:list):
    for category in categorylist:
        new_folder = pathlib.Path(os.path.join(clean_path, category))
        new_folder.mkdir(parents=True, exist_ok=True)

In [49]:
makeFolder(clean_path, categorylist)

In [50]:
import shutil

In [53]:
def moveFile(clean_path, trg_path, categorylist):
    folderlist = os.listdir(clean_path)
    filelist = os.listdir(trg_path)
    categorydict = {}

    for file in filelist:
        try:
            temp_list = file.split('_')
            assert temp_list[0] in categorydict
            categorydict[file] = temp_list[0]

        except:
            categorydict[file] = '기타'

    for key, value in categorydict.items():
        shutil.copy(trg_path+'/'+key, clean_path+'/'+value)

In [54]:
moveFile(clean_path, trg_path, categorylist)